In [14]:
from langchain_openai import ChatOpenAI
import pandas as pd
from dotenv import load_dotenv
from datasets import load_dataset, Dataset


from langchain_openai.chat_models import ChatOpenAI
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI

from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

load_dotenv()

True

In [15]:
# eval_df=pd.read_csv("tesla.csv")

In [22]:

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english")
eval_df = amnesty_qa["eval"].to_pandas()
eval_df

Repo card metadata block was not found. Setting CardData to empty.


,question,ground_truths,answer,contexts
0,What are the global implications of the USA Su...,[The global implications of the USA Supreme Co...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ..."
1,Which companies are the main contributors to G...,"[According to the Carbon Majors database, the ...","According to the Carbon Majors database, the m...","[In recent years, there has been increasing pr..."
2,Which private companies in the Americas are th...,[The largest private companies in the Americas...,"According to the Carbon Majors database, the l...",[The issue of greenhouse gas emissions has bec...
3,What action did Amnesty International urge its...,[Amnesty International urged its supporters to...,Amnesty International urged its supporters to ...,"[In the case of the Ogoni 9, Amnesty Internati..."
4,What are the recommendations made by Amnesty I...,[The recommendations made by Amnesty Internati...,Amnesty International made several recommendat...,"[In recent years, Amnesty International has fo..."
5,Who are the target audience of the two books c...,[The target audience of the two books created ...,The target audience of the two books created b...,"[In addition to children, parents, teachers, a..."
6,Which right guarantees access to comprehensive...,[The right that guarantees access to comprehen...,The right that guarantees access to comprehens...,[The right to truth is a fundamental human rig...
7,Who has the right to be fully informed about h...,[The victims of gross human rights violations ...,Everyone has the right to be fully informed ab...,"[In many cases, the identities of perpetrators..."
8,When can individuals be found guilty under Art...,[Individuals can be found guilty under Article...,Under Article 207.3 of the Russian Criminal Co...,[Article 207.3 of the Russian Criminal Code pe...
9,When does the prosecution consider statements ...,[The prosecution considers statements contrary...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...


In [17]:
# eval_df = load_dataset("harpreetsahota/ragas-example-dataset")["train"].to_pandas()

# eval_df = eval_df.rename(columns={"answer":"ground_truth"})
# eval_df


,question,ground_truth,contexts
0,Who was the first person to walk on the moon?,Buzz Aldrin was the first person to walk on th...,[Neil Armstrong was the first person to walk o...
1,What is the capital city of France?,Berlin is the capital city of France,"[Paris is the capital city of France, not Berl..."
2,Who is the current president of the United Sta...,Joe Biden is the current president of the Unit...,[Joe Biden is the current president of the Uni...
3,What is the square root of 81?,The square root of 81 is 9,"[The square root of 81 is indeed 9., This is a..."
4,"Who wrote the play ""Romeo and Juliet""?","William Shakespeare wrote the play ""Romeo and ...",[William Shakespeare is the playwright who wro...
5,What is the largest planet in our solar system?,Saturn is the largest planet in our solar system,[Jupiter is the largest planet in our solar sy...
6,Who is the founder of Microsoft?,Bill Gates is the founder of Microsoft,"[Bill Gates is the co-founder of Microsoft, al..."
7,What is the chemical symbol for hydrogen?,The chemical symbol for hydrogen is Hg,"[The chemical symbol for hydrogen is H, not Hg..."
8,Who is the current prime minister of Canada?,Justin Trudeau is the current prime minister o...,[Justin Trudeau is the current prime minister ...
9,What is the name of the world's largest desert?,The Sahara is the name of the world's largest ...,[The Sahara is the world's largest hot desert....


In [23]:
def get_answers(eval_df, agent):
    answers = []
    for index, row in eval_df.iterrows(): 
        context = row["contexts"]
        question = row["question"]
        
        prompt_input = f"""Answer the question: {question}
        Use the provided context to answer the question. The relevant contexts extracted from relevant documents are listed below:
        {context}"""
        
        result = agent.invoke(prompt_input)
        answers.append(result)
        
        return answers
    

def evaluate_answers(eval_df, answers, caption):

    result_df = eval_df.copy(deep=True)
    result_df["answer"] = pd.Series(answers)
    eval_dataset = Dataset.from_pandas(result_df)

    evaluation = evaluate(
        eval_dataset,
        metrics=[
            context_precision,
            faithfulness,
            answer_relevancy,
            context_recall,
        ],
    )
    print(f"Evaluation results - {caption}")
    print(evaluation)
    return evaluation


In [24]:

plain_llm = ChatOpenAI(model= "gpt-4o-mini", temperature=0.7)

plain_answers = get_answers(eval_df, plain_llm)
plain_answers = [answer.content for answer in plain_answers]
evaluate_answers(eval_df, plain_answers, "Plain OpenAI")


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   1%|▏         | 1/80 [00:01<01:54,  1.45s/it]No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  15%|█▌        | 12/80 [00:03<00:14,  4.84it/s]No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  30%|███       | 24/80 [00:07<00:14,  3.85it/s]No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  66%|██████▋   | 53/80 [00:16<00:0

Evaluation results - Plain OpenAI
{'context_precision': 0.9417, 'faithfulness': 0.9583, 'answer_relevancy': 0.0500, 'context_recall': 0.9500}


{'context_precision': 0.9417, 'faithfulness': 0.9583, 'answer_relevancy': 0.0500, 'context_recall': 0.9500}

In [25]:

model = ChatOpenAI(model= "gpt-4o-mini", temperature=0.7)
planner = load_chat_planner(model)
executor = load_agent_executor(model, [])
plan_execute_agent = agent = PlanAndExecute(planner=planner, executor=executor)


plan_and_execute_answers = get_answers(eval_df, plan_execute_agent)
plan_and_execute_answers = [ answer["output"] for answer in plan_and_execute_answers]
evaluate_answers(eval_df, plan_and_execute_answers, "Plan and Execute")

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   2%|▎         | 2/80 [00:02<01:17,  1.00it/s]No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  30%|███       | 24/80 [00:08<00:16,  3.49it/s]No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  41%|████▏     | 33/80 [00:10<00:10,  4.50it/s]No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  45%|████▌     | 36/80 [00:10<00:07,  6.14it/s]No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  64%|██████▍   | 51/80 [00:15<00:07,  3.85it/s]No statements were generated from the answer.
No statements were g

Evaluation results - Plan and Execute
{'context_precision': 0.9417, 'faithfulness': 0.8000, 'answer_relevancy': 0.0972, 'context_recall': 0.9500}


{'context_precision': 0.9417, 'faithfulness': 0.8000, 'answer_relevancy': 0.0972, 'context_recall': 0.9500}